In [ ]:
import pandas as pd
import spacy

df_train= pd.read_csv("politicES_phase_2_train_public.csv", on_bad_lines='skip')
df_test= pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')

In [ ]:
# Agrupar los tweets por 'label' y 'ideology_multiclass' y luego concatenar los tweets
df_train= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()
df_test= df_test.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()

In [ ]:
df_train.drop('label', axis=1, inplace=True)
df_test.drop('label', axis=1, inplace=True)

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load('es_core_news_lg')

def preprocess_text(tweet):
    # Procesa el tweet usando el modelo de spaCy
    doc = nlp(tweet)
    # Lematiza y elimina stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [ ]:
df_train['clean_tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['clean_tweet'] = df_test['tweet'].apply(preprocess_text)

In [ ]:
df_train.to_csv('train_tokenized.csv')
df_test.to_csv('test_tokenized.csv')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_train= pd.read_csv('/content/drive/My Drive/TFM/train_tokenized.csv')
df_test= pd.read_csv('/content/drive/My Drive/TFM/test_tokenized.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_train['ideology_multiclass'] = label_encoder.fit_transform(df_train['ideology_multiclass'])
df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])

In [4]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import spacy

# Cargar el modelo español con vectores
nlp = spacy.load('es_core_news_lg')
docs = [nlp(text) for text in df_train['tweet']]


In [6]:
def get_document_vector(doc):
    tokens = [token for token in doc if token.has_vector and not token.is_stop]
    if len(tokens) == 0:
        return np.zeros((nlp.vocab.vectors_length,))
    return np.mean([token.vector for token in tokens], axis=0)

doc_vectors_train = np.array([get_document_vector(doc) for doc in docs])

In [7]:

y = df_train['ideology_multiclass']

# Dividir el conjunto de entrenamiento para validación (10% para test)
X_train, X_val, y_train, y_val = train_test_split(doc_vectors_train, y, test_size=0.3, random_state=42)

# Definir varios modelos para probar
modelos = {
    'XGB': xgb.XGBClassifier(),
    'Regresión Logística': LogisticRegression(),
    'SVM': SVC(),

}

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_val)
    macro_f1 = f1_score(y_val, y_pred, average='macro')
    print(f"{nombre} - Macro F1 Score: {macro_f1}")

XGB - Macro F1 Score: 0.8262498401303279


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Regresión Logística - Macro F1 Score: 0.7425758158399791
SVM - Macro F1 Score: 0.5494444641895136


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

# Definir el modelo y los parámetros para GridSearchCV
regresor = LogisticRegression(random_state=42, max_iter=1000)
param_grid = {
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'lbfgs'],
    'C': [1, 10, 100]
}

# Crear un GridSearchCV
clf = GridSearchCV(regresor, param_grid, scoring='f1_macro', cv=5, verbose=1, n_jobs=-1)

# Ajustar el modelo
clf.fit(X_train, y_train)

# Mejores parámetros y mejor score
print("Mejores parámetros:", clf.best_params_)
print("Mejor puntuación de cross-validation (macro F1):", clf.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

Mejores parámetros: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Mejor puntuación de cross-validation (macro F1): 0.8036175106074358


In [13]:
docs_test = [nlp(text) for text in df_test['tweet']]

In [14]:
X_test = np.array([get_document_vector(doc) for doc in docs_test])
y_test = df_test['ideology_multiclass']
y_pred = clf.best_estimator_.predict(X_test)

macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1 en el conjunto de prueba:", macro_f1)
print(classification_report(y_test, y_pred))

Macro F1 en el conjunto de prueba: 0.5281741611074598
              precision    recall  f1-score   support

           0       0.50      0.56      0.53       117
           1       0.57      0.60      0.59       210
           2       0.55      0.59      0.57       153
           3       0.71      0.30      0.42        67

    accuracy                           0.56       547
   macro avg       0.58      0.52      0.53       547
weighted avg       0.57      0.56      0.55       547

